In [ ]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id', 
             'air_reserved_visitors', 'hpg_reserved_visitors','reserved_visitors']

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l1/kfold' % DataBaseDir
InputDir2 = '%s/meta/kfold' % DataBaseDir
OutputDir = '%s/l2/kfold' % DataBaseDir
kfold = 5
# InputCols = ['lgb_l2', 'xgb_rmse','lgb_huber', 'lgb_fair', 'etr', 'en', 'rf',  'gbr_ls', 'gbr_huber', 'gbr_lad', 'lassolars', 'rgf']
InputCols = ['lgb_l2', 'xgb_rmse']

strategy = 'en'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
# join_keys = ['air_store_id', 'visit_date']
# l0_cols = ['air_store_id', 'visit_date', 'air_genre_name', 'dow', 'hol_days', 'holiday_flg', 'dow', 
#            'visitor_rolling_10_5_mean', 'visitor_rolling_15_10_mean', 'visitor_rolling_20_15_mean', 
#            'visitor_rolling_25_20_mean', 'exp_smooth', 'air_store_id_dow_mean']
meta_feats = ['nn', 'knn_2', 'knn_4', 'knn_8', 'knn_16', 'knn_32', 'knn_64', 'knn_128', 'knn_256', 'knn_512', 'knn_1024']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    for i in range(len(InputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid[InputCols[i]] = valid[InputCols[i]]
            FoldHoldout[InputCols[i]] = holdout[InputCols[i]]
            FoldTest[InputCols[i]] = test[InputCols[i]]
    for t in meta_feats:
        FoldInputDir = '%s/%s' % (InputDir2, fold)
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[t] = valid[t]
        FoldHoldout[t] = holdout[t]
        FoldTest[t] = test[t]

#     FoldInputDir = '%s/l0/kfold/%s' % (DataBaseDir, fold)
#     valid = pd.read_csv('%s/valid.csv' % (FoldInputDir), parse_dates= ['visit_date']).reset_index(drop= True)
#     holdout = pd.read_csv('%s/holdout.csv' % (FoldInputDir), parse_dates= ['visit_date']).reset_index(drop= True)
#     test = pd.read_csv('%s/test.csv' % (FoldInputDir), parse_dates= ['visit_date']).reset_index(drop= True)
#     FoldValid = FoldValid.merge(valid[l0_cols], on= ['air_store_id', 'visit_date'])
#     FoldHoldout = FoldHoldout.merge(holdout[l0_cols], on= ['air_store_id', 'visit_date'])
#     FoldTest = FoldTest.merge(test[l0_cols], on= ['air_store_id', 'visit_date'])
    ###
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ TestData')
print(test_dfs[0].head(5))
print('-------------------------------------')
# sys.exit(1)
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in drop_cols]
    # train
    model = linear_model.ElasticNet(alpha= 0.0001, l1_ratio= 0.2, max_iter= 400, tol= 1e-4, selection= 'random', random_state= 2017)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors    lgb_l2  xgb_rmse        nn  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.380776  3.392094  3.559292   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.425436  3.466866  3.559292   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  3.215661  3.259613  3.352624   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  3.278749  3.345982  3.507653   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.152712  3.174966  3.375907   

      knn_2     knn_4     knn_8    knn_16    knn_32    knn_64   knn_128  \
0  3.481122  3.052317  2.852771  2.909590  2.932775  2.776691  2.707737   
1  3.091042  2.902858  2.961209  2.909590  2.932775  2.839447  2.704730   
2  3.215666  2.922207  2.764926  2.913624  2.938812  2.805807  2.715027   
3  3.113268  2.926794  2.790034  2.789120  2.961501  3.036717  2.660556   
4  2.931816  3.073741  2.801453  2.959925  3.014474  3.036717  2.702637   

  

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.505141, holdout score 0.495234, valid length 45371
